In [ ]:
import itertools
import joblib
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import yaml

# import plot_confusion_matrix()
from src.report.visualize import plot_confusion_matrix

In [ ]:
# Go to project root folder
%cd ..

# Config

In [ ]:
# Read config
import pprint

with open('params.yaml') as conf_file:
    config = yaml.safe_load(conf_file)

pprint.pprint(config)

# Load dataset

In [ ]:
# Get data 

import pandas as pd
from sklearn.datasets import load_iris

data = load_iris(as_frame=True)
dataset = data.frame
dataset.head()

In [ ]:
# print labels for target values 

[print(f'{target}: {label}') for target, label in zip(data.target.unique(), data.target_names)]

In [ ]:
# feature names

dataset.columns = [colname.strip(' (cm)').replace(' ', '_') for colname in dataset.columns.tolist()]

feature_names = dataset.columns.tolist()[:4]
feature_names

In [ ]:
# Save raw data
dataset.to_csv(config['data']['dataset_csv'], index=False)

# Features engineering

In [ ]:
dataset['sepal_length_to_sepal_width'] = dataset['sepal_length'] / dataset['sepal_width']
dataset['petal_length_to_petal_width'] = dataset['petal_length'] / dataset['petal_width']

dataset = dataset[[
    'sepal_length', 'sepal_width', 'petal_length', 'petal_width',
#     'sepal_length_in_square', 'sepal_width_in_square', 'petal_length_in_square', 'petal_width_in_square',
    'sepal_length_to_sepal_width', 'petal_length_to_petal_width',
    'target'
]]

In [ ]:
dataset.head()

In [ ]:
# Save features
dataset.to_csv(config['data']['features_path'], index=False)

# Split dataset

In [ ]:
train_dataset, test_dataset = train_test_split(
    dataset, test_size=config['data']['test_size'],
    random_state=config['base']['random_state']
)
train_dataset.shape, test_dataset.shape

In [ ]:
# Save train and test sets
train_dataset.to_csv(config['data']['trainset_path'])
test_dataset.to_csv(config['data']['testset_path'])

# Train

In [ ]:
# Get X and Y

y_train = train_dataset.loc[:, 'target'].values.astype('int32')
X_train = train_dataset.drop('target', axis=1).values.astype('float32')

In [ ]:
# Create an instance of Logistic Regression Classifier CV and fit the data

logreg = LogisticRegression(
    **config['train']['clf_params'],
    random_state=config['base']['random_state']
)
logreg.fit(X_train, y_train)

In [ ]:
joblib.dump(logreg, config['train']['model_path'])

# Evaluate

In [ ]:
# Get X and Y

y_test = test_dataset.loc[:, 'target'].values.astype('int32')
X_test = test_dataset.drop('target', axis=1).values.astype('float32')

In [ ]:
prediction = logreg.predict(X_test)
cm = confusion_matrix(prediction, y_test)
f1 = f1_score(y_true = y_test, y_pred = prediction, average='macro')

In [ ]:
# f1 score value
f1

In [ ]:
# Save metrics
metrics = {
    'f1': f1
}

with open(config['reports']['metrics_file'], 'w') as mf:
    json.dump(
        obj=metrics,
        fp=mf,
        indent=4
    )

In [ ]:
cm_plot = plot_confusion_matrix(cm, data.target_names, normalize=False)

In [ ]:
# Save confusion matrix image
cm_plot.savefig(config['reports']['confusion_matrix_image'])